# Test Valuation Engines

This notebook walks through:

1. Loading historical SOFR fixings
2. Building a simple two-index yield curve
3. Valuing various products (bullet, IBOR cashflow, OIS cashflow, futures, swaps)

---

## 1. Imports & Setup

In [30]:
import pandas as pd
from fixedincomelib.data import DataCollection, Data1D, build_yc_data_collection
from fixedincomelib.date import Date
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.valuation import IndexManager
from fixedincomelib.valuation import ValuationEngineRegistry
from fixedincomelib.product import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    ProductPortfolio )
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor

## 2. Load SOFR Fixings

In [31]:
IndexManager.instance()

## 3. Build Dummy Yield Curve

In [32]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [33]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [34]:
dc.dataMap

{('RFR FUTURE',
  'SOFR-FUTURE-3M'): Data1D(type='RFR FUTURE', conv='SOFR-FUTURE-3M', points=12),
 ('RFR SWAP',
  'USD-SOFR-OIS'): Data1D(type='RFR SWAP', conv='USD-SOFR-OIS', points=14)}

In [35]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT"
}]


In [36]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [37]:
from fixedincomelib.date import addPeriod, moveToBusinessDay

StartDate = "2025-09-24"
print(Date(StartDate).ISO(),yc.forward(index="SOFR-1B", effectiveDate=Date(StartDate),termOrTerminationDate="3M"))
for i in range(20):
    new_date = addPeriod(StartDate,"3M")
    print(new_date.ISO(), 
          yc.forward(index="SOFR-1B", effectiveDate=new_date,termOrTerminationDate="3M"),
          yc.discountFactor(index='SOFR-1B', to_date=new_date))
    StartDate = moveToBusinessDay(input_date=new_date,biz_conv="F",hol_conv="USGS")
    

2025-09-24 0.04300000000000149
2025-12-24 0.04199999999999715 0.9892474300176138
2026-03-24 0.041000000000003596 0.9789682632534533
2026-06-24 0.039999999999994436 0.9688172118009162
2026-09-24 0.03920000000000031 0.9590139580079474
2026-12-24 0.038399999999999324 0.9496044331910609
2027-03-24 0.03760000000000305 0.9405749140165026
2027-06-24 0.03679999999999954 0.9316230516270897
2027-09-24 0.036238783848969724 0.9229432827986368
2027-12-24 0.03683328365099615 0.9145744187699928
2028-03-24 0.03502627489337901 0.9061377008434798
2028-06-24 0.03524290835387669 0.8981046536634887
2028-09-26 0.030629224095240645 0.8899254405042977
2028-12-26 0.030969548807409986 0.8830882294981248
2029-03-26 0.030296297746379337 0.8763035481222056
2029-06-26 0.030395124365989748 0.869570992796476
2029-09-26 0.0351277721107933 0.862868538128233
2029-12-26 0.03551808068980211 0.8552741143766508
2030-03-26 0.034745948500893374 0.8477465319451191
2030-06-26 0.03486070002429162 0.8402852025385662
2030-09-26 0.

In [38]:
comp = yc.retrieveComponent("SOFR-1B")

In [39]:
yc.forwardRateGradientWrtModelParameters(index="SOFR-1B", start_time= "2025-09-24", end_time="2026-02-24", scaler =1, accumulate=False)

In [40]:
yc.gradient_

array([0.61075992, 0.40717328, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [41]:
yc.discountFactorGradientWrtModelParameters(index="SOFR-1B", to_date="2045-12-24", scaler =1, accumulate=False)

In [42]:
yc.gradient_

array([-0.10815323, -0.10815323, -0.10815323, -0.10815323, -0.10815323,
       -0.10815323, -0.10815323, -0.10815323, -0.10815323, -0.10815323,
       -0.10815323, -0.10815323, -0.43261292, -0.43261292, -0.43261292,
       -0.43261292, -0.43501632, -0.43141122, -0.43141122, -2.1630646 ,
       -2.1642663 , -0.10695153, -0.        , -0.        , -0.        ,
       -0.        ])

In [43]:
yc.discountFactorGradientWrtModelParameters(index="SOFR-1B", to_date="2045-12-24", scaler =-1, accumulate=True)

In [44]:
yc.gradient_

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [45]:
comp.state_variables


[0.04324318651958291,
 0.0417810314461517,
 0.04169306522518325,
 0.040681314616304115,
 0.03944047105275002,
 0.038216851219245986,
 0.03825206853208043,
 0.03744199188020869,
 0.036435771528606595,
 0.037070235188509555,
 0.03561871936079189,
 0.03591010968343242,
 0.03085027493566167,
 0.03536131624473848,
 0.04059335164181802,
 0.03970815567485271,
 0.04301284967241085,
 0.041324936634231535,
 0.04205524738132715,
 0.04213061711740104,
 0.0452662264154874,
 0.04628329880739514,
 0.049198571717585314,
 0.045622198964126226,
 0.050104543919094656,
 0.053622859201824837]

In [46]:
comp.pillar_dates


[Date(24,12,2025),
 Date(24,3,2026),
 Date(24,6,2026),
 Date(24,9,2026),
 Date(24,12,2026),
 Date(24,3,2027),
 Date(24,6,2027),
 Date(24,9,2027),
 Date(24,12,2027),
 Date(24,3,2028),
 Date(24,6,2028),
 Date(24,9,2028),
 Date(24,9,2029),
 Date(24,9,2030),
 Date(24,9,2031),
 Date(24,9,2032),
 Date(26,9,2033),
 Date(25,9,2034),
 Date(24,9,2035),
 Date(24,9,2040),
 Date(25,9,2045),
 Date(26,9,2050),
 Date(24,9,2055),
 Date(24,9,2065),
 Date(24,9,2075),
 Date(24,9,2085)]

In [47]:
comp.pillar_times_to_date


[0.25,
 0.5,
 0.75,
 1.0,
 1.25,
 1.5,
 1.75,
 2.0,
 2.25,
 2.5,
 2.75,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.005555555555556,
 9.002777777777778,
 10.0,
 15.0,
 20.002777777777776,
 25.005555555555556,
 30.0,
 40.0,
 50.0,
 60.0]

In [48]:
comp.pillar_nodes

[PillarNode(node_id='RFR FUTURE SOFR-FUTURE-3M September 24th, 2025 x December 24th, 2025', pillar_index=0, pillar_time=0.25, pillar_date=Date(24,12,2025), start_date=Date(24,9,2025), end_date=Date(24,12,2025), instrument=<fixedincomelib.product.linear_products.ProductRfrFuture object at 0x00000233E7FC0350>, state_value=0.04324318651958291),
 PillarNode(node_id='RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 x March 24th, 2026', pillar_index=1, pillar_time=0.5, pillar_date=Date(24,3,2026), start_date=Date(24,12,2025), end_date=Date(24,3,2026), instrument=<fixedincomelib.product.linear_products.ProductRfrFuture object at 0x00000233E7FC04D0>, state_value=0.0417810314461517),
 PillarNode(node_id='RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x June 24th, 2026', pillar_index=2, pillar_time=0.75, pillar_date=Date(24,6,2026), start_date=Date(24,3,2026), end_date=Date(24,6,2026), instrument=<fixedincomelib.product.linear_products.ProductRfrFuture object at 0x00000233E7F97C50>, state_value=0.04169

In [49]:
yc.jacobian()

array([[-100.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,  -98.924743  ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,  -97.89682633,    0.        ,
           0.        ,    0.        , 

In [50]:
yc._jacobian_row_labels

['RFR FUTURE SOFR-FUTURE-3M September 24th, 2025 x December 24th, 2025',
 'RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 x March 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x June 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M June 24th, 2026 x September 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M September 24th, 2026 x December 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M December 24th, 2026 x March 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M March 24th, 2027 x June 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M June 24th, 2027 x September 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M September 24th, 2027 x December 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M December 24th, 2027 x March 24th, 2028',
 'RFR FUTURE SOFR-FUTURE-3M March 24th, 2028 x June 24th, 2028',
 'RFR FUTURE SOFR-FUTURE-3M June 24th, 2028 x September 24th, 2028',
 'RFR SWAP USD-SOFR-OIS September 24th, 2029',
 'RFR SWAP USD-SOFR-OIS September 24th, 2030',
 'RFR SWAP USD-SOFR-OIS September 24th, 2031',
 'RFR SWAP USD-SOFR-OIS Septemb


## 4. Value Bullet Cashflow


In [51]:
# Bullet cashflow PV
bullet = ProductBulletCashflow("2028-09-24", "USD", 1000000, "LONG")
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    bullet
)
ve.calculateValue()
print("Bullet CF PV:", ve.value)

Bullet CF PV: ['USD', np.float64(890077.9782674118)]


## 5. Value IBOR Cashflow

In [53]:
ibor_cf = ProductIborCashflow(
    startDate="2025-09-27",
    endDate="2025-12-27",
    index="USD-LIBOR-BBA-3M",
    spread=0.0,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {"FUNDING INDEX": "SOFR-1B"}, ibor_cf)
ve.calculateValue()
print("Ibor CF PV:", ve.value)

AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

## 6. Value OIS Cashflow

In [54]:
ois_cf = ProductOvernightIndexCashflow(
    effectiveDate="2025-09-24",
    termOrEnd="6M",
    index="SOFR-1B",
    compounding="COMPOUND",
    spread=0.0,
    notional=1000000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    ois_cf
)
ve.calculateValue()
print("OIS CF PV:", ve.value)

OIS CF PV: ['USD', 21483.573611110973]


## 7. Value Futures

In [ ]:
# IBOR Future PV
future = ProductFuture(
    effectiveDate="2025-06-27",
    index="USD-LIBOR-BBA-3M",
    strike=99.5,
    notional=1_000_000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {"FUNDING INDEX": "SOFR-1B"}, future)
ve.calculateValue()
print("Future PV:", ve.value)

AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

In [55]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="AVERAGE",
    strike=95.9,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

RFR Future PV: ['USD', np.float64(175388.94039866852)]


In [56]:
ve.calculateFirstOrderRisk()
print(ve.firstOrderRisk_)

[99956152.76490031        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]


## 8. Value a Portfolio

In [57]:
# Simple portfolio of bullet + IBOR CF
portfolio = ProductPortfolio([bullet, ois_cf], weights=[0.5, 0.5])
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    portfolio
)
ve.calculateValue()
print("Portfolio PV:", ve.value)

Portfolio PV: ['USD', np.float64(911561.5518785228)]


## 9. Value Swaps (IBOR & OIS)

In [ ]:
# IBOR Swap
ibor_swap = ProductIborSwap(
    effectiveDate="2025-06-27",
    maturityDate="2026-06-27",
    frequency="6M",
    iborIndex="USD-LIBOR-BBA-3M",
    spread=0.0,
    fixedRate=0.015,
    notional=1_000_000,
    position="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ibor_swap
)
ve.calculateValue()
print("IBOR Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())



AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

In [58]:
# OIS Swap
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-09-24",
    maturityDate="2045-09-24",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.045,
    notional=1000000,
    position="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ois_swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(62489.085057563265)] Par Rate: 0.04050004909458539


In [59]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk_

array([ -262844.49348661,  -260066.71570883,  -260084.98202536,
        -257365.62573854,  -257234.57942795,  -249242.79644455,
        -249250.52490941,  -249250.52490941,  -246507.33237442,
        -238816.1383511 ,  -233755.84452175,  -236272.89369076,
        -912493.42047604,  -875828.66247422,  -838127.03229097,
        -806432.96812283,  -773664.79559475,  -729834.33106029,
        -698151.30026053, -3087845.01317516, -2459105.56955042,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [60]:
from fixedincomelib.utilities import createValueReport

pv_only = createValueReport({"FUNDING INDEX": "SOFR-1B"}, yc, ois_swap, request="value")

In [61]:
print(pv_only)

62489.085057563265
